## load dataset

In [106]:
from datasets import load_dataset, load_metric 

dataset = load_dataset('csv', data_files={'train':["../dataset/MMLU_train_set.csv"], 'test':['../dataset/MMLU_dev_set.csv']})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [107]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'A', 'B', 'C', 'D', 'answer'],
        num_rows: 99842
    })
    test: Dataset({
        features: ['question', 'A', 'B', 'C', 'D', 'answer'],
        num_rows: 285
    })
})


In [108]:
dataset["train"][0]

{'question': "Davis decided to kill Adams. He set out for Adams's house. Before he got there he saw Brooks, who resembled Adams. Thinking that Brooks was Adams, Davis shot at Brooks. The shot missed Brooks but wounded Case, who was some distance away. Davis had not seen Case. In a prosecution under a statute that proscribes any attempt to commit murder, the district attorney should indicate that the intended victim(s) was/were",
 'A': 'Adams only.',
 'B': 'Brooks only.',
 'C': 'Case only.',
 'D': 'Adams and Brooks',
 'answer': 'B'}

In [109]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))


In [110]:
show_random_elements(dataset["train"])

,question,A,B,C,D,answer
0,"It was something she had dreamed of since she was five. Finally, after years of training and intensive workouts, Deborah Duffey was going to compete in her first high school basketball game. The goals of becoming an outstanding player and playing college ball were never far from Deborah's mind. The game was against Mills High School. With 1 minute and 42 seconds left in the game, Deborah's team led by one point. A player of Mills had possession of the ball,and Deborah ran to guard against her. As Deborah was running to block the player, her knee went out and she fell down on the court in burning pain. Just like that, Deborah's season was over. After suffering the bad injury, Deborah found that, for the first time in her life, she was in a situation beyond her control. Game after game, she could do nothing but sit on the sidelines watching others play the game that she loved so much. Injuries limited Deborah's time on the court as she hurt her knees three more times in the next five years. She had to spend countless hours in a physical clinic to receive treatment. Her frequent visits there gave her a passion and respect for the profession. And Deborah began to see a new light in her life. Currently as a senior in college, Deborah focuses on getting a degree in physical treatment. After she graduates, Deborah plans to use her knowledge to educate people how to best take care of their bodies and cope with the feelings of hopelessness that she remembers so well. What is the best title for this passage?",A Painful Mistake,A Great Adventure,A Lifelong Punishment,A New Direction in Life,D
1,Which best determines the health of a lake used as a source of freshwater?,its depth and width,its temperature and pH,its location and depth,its temperature and depth,B
2,"According to decades-long research, women who took low to moderate daily doses of aspirin had a reduced death rate, especially from heart disease. The research, based on data from a major trial that has tracked almost 80,000 women since 1976, found that women who reported using aspirin on a regular basis had a 25 percent lower risk of death from any cause than women who didn't take the drug. The risk of death from cardiovascular disease was 38 percent lower for aspirin users, and there was also a 12 percent reduction in cancer deaths that took effect after a decade of aspirin use, the researchers found in their report based on the Nurses' Health Study. However, an accompanying editorial in the journal cautioned that the results were open to debate and far from definitive. The dissenting editorial was based on results of an earlier trial by the Women's Health Study, which followed almost 40,000 women for 11 years and found no reduction in overall deaths. Therefore, the new findings ""cannot overcome the accumulated evidence that aspirin is not particularly effective for the primary prevention of death from cardiovascular disease in women."" ""This is a complicated issue,"" said Dr. Andrew T. Chan, leading author of the new report. ""We understand that aspirin has potential health benefits, but who would aspirin therapy be appropriate for?"" There are ""areas of disagreement that need further study"" before that question can be answered, Chan said. But there is information from the two large studies and other trials that can help guide women and their physicians, he said. And anyone who is thinking about daily aspirin ""should really talk with doctors about the benefits and risks."" Which of the following questions hasn't been solved according to Dr. Chan?",Whether aspirin is beneficial for health?,Which of the two researches is really reliable?,Who will do the further study in this area?,Who are proper for this treatment?,D
3,"For years, business people in Western Europe were worried. They knew they could not compete against business from the U.S. The United States is much larger and had many more resources than any Western European countries. Some Eur

In [111]:
def show_one(example):
    print(f"Question: {example['question']}")
    print(f"  A - {example['A']}")
    print(f"  B - {example['B']}")
    print(f"  C - {example['C']}")
    print(f"  D - {example['D']}")
    print(f"\nGround truth: option {example['answer']}")

In [112]:
show_one(dataset["train"][0])

Question: Davis decided to kill Adams. He set out for Adams's house. Before he got there he saw Brooks, who resembled Adams. Thinking that Brooks was Adams, Davis shot at Brooks. The shot missed Brooks but wounded Case, who was some distance away. Davis had not seen Case. In a prosecution under a statute that proscribes any attempt to commit murder, the district attorney should indicate that the intended victim(s) was/were
  A - Adams only.
  B - Brooks only.
  C - Case only.
  D - Adams and Brooks

Ground truth: option B


## preprocessing the data

In [113]:
from transformers import AutoTokenizer

model_ckpt = 'allenai/longformer-base-4096'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True)

In [114]:
options={'A':0, 'B':1, 'C':2, 'D':3}

def preprocess_function(examples):
    # Repeat each questions four times to go with the four possibilities of second sentences.
    first_sentences = [[context] * 4 for context in examples["question"]]
    
    # Grab all option sentences possible for each question.
    second_sentences = [[f"{examples[option][i]}" for option in options] for i in range(len(examples['question']))]
    
    # Flatten everything
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    
    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)

    # label idx
    encoded_labels = [options[f'{label}'] for label in examples['answer']]
    
    # Un-flatten
    output_dict={k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}
    output_dict['label'] = encoded_labels
    return output_dict

In [115]:
examples = dataset["train"][:5]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

5 4 [95, 96, 95, 95]


In [116]:
idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]

["<s>Peter sued Don for breach of contract. The court admitted testimony by Peter that Don and his wife quarreled frequently, a fact of no consequence to the lawsuit. Don seeks to testify in response that he and his wife never quarreled. The court</s></s>must permit Don to answer if he had objected to Peter's testimony.</s>",
 "<s>Peter sued Don for breach of contract. The court admitted testimony by Peter that Don and his wife quarreled frequently, a fact of no consequence to the lawsuit. Don seeks to testify in response that he and his wife never quarreled. The court</s></s>may permit Don to answer, whether or not he had objected to Peter's testimony. </s>",
 "<s>Peter sued Don for breach of contract. The court admitted testimony by Peter that Don and his wife quarreled frequently, a fact of no consequence to the lawsuit. Don seeks to testify in response that he and his wife never quarreled. The court</s></s>may permit Don to answer only if he had objected to Peter's testimony.</s>",

In [117]:
encoded_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/99842 [00:00<?, ? examples/s]

Map:   0%|          | 0/285 [00:00<?, ? examples/s]

KeyError: '3'

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained(model_ckpt)

In [ ]:
model_name = model_ckpt.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [95]:
encoded_datasets["train"][0]

{'question': 'What is true for a type-Ia ("type one-a") supernova?',
 'A': 'This type occurs in binary systems.',
 'B': 'This type occurs in young galaxies.',
 'C': 'This type produces gamma-ray bursts.',
 'D': 'This type produces high amounts of X-rays.',
 'answer': 'A',
 'input_ids': [[0,
   2264,
   16,
   1528,
   13,
   10,
   1907,
   12,
   100,
   102,
   6697,
   12528,
   65,
   12,
   102,
   8070,
   2422,
   38823,
   116,
   2,
   2,
   713,
   1907,
   11493,
   11,
   32771,
   1743,
   4,
   2],
  [0,
   2264,
   16,
   1528,
   13,
   10,
   1907,
   12,
   100,
   102,
   6697,
   12528,
   65,
   12,
   102,
   8070,
   2422,
   38823,
   116,
   2,
   2,
   713,
   1907,
   11493,
   11,
   664,
   30948,
   4,
   2],
  [0,
   2264,
   16,
   1528,
   13,
   10,
   1907,
   12,
   100,
   102,
   6697,
   12528,
   65,
   12,
   102,
   8070,
   2422,
   38823,
   116,
   2,
   2,
   713,
   1907,
   9108,
   42685,
   12,
   5022,
   28740,
   4,
   2],
  [0,
   2

In [94]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys}]

Dataset({
    features: ['question', 'A', 'B', 'C', 'D', 'answer', 'input_ids', 'attention_mask', 'label'],
    num_rows: 453
})

In [89]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
print(features[0])
batch = DataCollatorForMultipleChoice(tokenizer)(features)

{'input_ids': [[0, 2264, 16, 1528, 13, 10, 1907, 12, 100, 102, 6697, 12528, 65, 12, 102, 8070, 2422, 38823, 116, 2, 2, 713, 1907, 11493, 11, 32771, 1743, 4, 2], [0, 2264, 16, 1528, 13, 10, 1907, 12, 100, 102, 6697, 12528, 65, 12, 102, 8070, 2422, 38823, 116, 2, 2, 713, 1907, 11493, 11, 664, 30948, 4, 2], [0, 2264, 16, 1528, 13, 10, 1907, 12, 100, 102, 6697, 12528, 65, 12, 102, 8070, 2422, 38823, 116, 2, 2, 713, 1907, 9108, 42685, 12, 5022, 28740, 4, 2], [0, 2264, 16, 1528, 13, 10, 1907, 12, 100, 102, 6697, 12528, 65, 12, 102, 8070, 2422, 38823, 116, 2, 2, 713, 1907, 9108, 239, 5353, 9, 1577, 12, 24192, 4, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [93]:
[tokenizer.decode(batch["input_ids"][1][i].tolist()) for i in range(4)]

['<s>If you know both the actual brightness of an object and its apparent brightness from your location then with no other information you can estimate:</s></s>Its speed relative to you</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s>If you know both the actual brightness of an object and its apparent brightness from your location then with no other information you can estimate:</s></s>Its composition</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s>If you know both the actual brightness of an object and its apparent brightness from your location then with no other information you can estimate:</s></s>Its size</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [92]:
show_one(dataset["train"][8])

Question: What is the second most common element in the solar system?
  A - Iron
  B - Hydrogen
  C - Methane
  D - Helium

Ground truth: option D


In [96]:
len(features)

10